In [1]:
import numpy as np

X = np.array([
  [1,1,0,1],
  [1,1,1,1],
  [1,0,1,1],
  [0,0,0,1]
])
                
y = np.array([[1],[0],[1],[0]])

In [11]:
np.random.seed(1)
class NN:
  def __init__(self, X, y, hidden_layers):
    self.X = np.array(X)
    self.y = np.array(y)
    self.schema = [X.shape[1]] + hidden_layers + [y.shape[1]]
    self.schema_len = range(len(self.schema[:-1]))
    for i, layer in enumerate(self.schema[:-1]):
      setattr(self, f'W{i}', np.random.uniform(-1, 1, (layer, self.schema[i+1])))

      
  def nonlin(self, x, deriv=False):
    if(deriv==True):
      return x * (1 - x)
    return 1/ (1 + np.exp(-x))
  
  
  def forward(self, X=None):
    if X:
      self.l0 = np.array(X)
    else:
      self.l0 = self.X
    for i in self.schema_len:
      l, W = (getattr(self, j) for j in f'l{i} W{i}'.split())
      setattr(self, f'l{i+1}', self.nonlin(l.dot(W)))
    return getattr(self, f'l{len(self.schema_len)}')
    
    
  def backward(self):

    # calculation loss
    for i in reversed(self.schema_len):
      i += 1
      l = getattr(self, f'l{i}')
      if i == len(self.schema_len):
        error = self.y - l
        self.error = np.mean(abs(error))
      else:
        delta, W = (getattr(self, j) for j in f'l{i}delta W{i}'.split())
        error = delta.dot(W.T)
      setattr(self, f'l{i-1}delta', error * self.nonlin(l, deriv=True))
      
    # adjusting weights
    for i in reversed(self.schema_len):
      l, W, delta = (getattr(self, j) for j in f'l{i} W{i} l{i}delta'.split())
      setattr(self, f'W{i}', W + l.T.dot(delta))
    
    
  def train(self, epochs=1000, print_nth_epoch=100):
    for j in range(epochs):
      self.forward()
      self.backward()
      if print_nth_epoch and not j % print_nth_epoch:
        print(f'Error: {round(self.error,4)}')
       
      
  def predict(self, x):
    self.forward(x)

In [12]:
nn = NN(X, y, [100,50,25,10])

In [13]:
nn.train()

Error: 0.4979
Error: 0.2359
Error: 0.0379
Error: 0.0249
Error: 0.0196
Error: 0.0166
Error: 0.0146
Error: 0.0132
Error: 0.0121
Error: 0.0112
